In [6]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import reuters
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np


# genre

In [37]:
rating = pd.read_csv("/content/rating.csv")
rating.head(20)

,title,rating,format,genre
0,가난을 등에 업은 소녀,9.11,story,pure
1,가담항설,9.98,story,fantasy
2,가령의 정체불명 이야기,9.95,omnibus,drama
3,가비지타임,9.96,story,sports
4,가상&RPG,9.94,story,fantasy
5,가슴털 로망스,9.93,story,comic
6,가우스전자 시즌1~2,9.96,story,comic
7,가우스전자 시즌3~4,9.97,episode,comic
8,가족같은 XX,9.96,story,pure
9,가족의 초상,9.88,episode,drama


In [38]:
mapping = {'pure':0, 'fantasy':1, 'drama':2, 'sports':3, 'comic':4, 'thrill':5, 'action':6, 'daily':7, 'sensibility':8, 'historical':9}
mapping2 = {'story':0, 'omnibus':1, 'episode':2}

a = rating['format'].map(mapping2)
rating['format'] = a
b = rating['genre'].map(mapping)
rating['genre'] = b

rating.head(20)


,title,rating,format,genre
0,가난을 등에 업은 소녀,9.11,0,0
1,가담항설,9.98,0,1
2,가령의 정체불명 이야기,9.95,1,2
3,가비지타임,9.96,0,3
4,가상&RPG,9.94,0,1
5,가슴털 로망스,9.93,0,4
6,가우스전자 시즌1~2,9.96,0,4
7,가우스전자 시즌3~4,9.97,2,4
8,가족같은 XX,9.96,0,0
9,가족의 초상,9.88,2,2


In [39]:
rating.to_csv("./rating_aa.csv")

In [23]:
df = pd.read_csv("/content/drive/MyDrive/reformedData (4).csv")

mapping = {'pure':0, 'fantasy':1, 'drama':2, 'sports':3, 'comic':4, 'thrill':5, 'action':6, 'daily':7, 'sensibility':8, 'historical':9}
mapping2 = {'story':0, 'omnibus':1, 'episode':2}

a = df['55'].map(mapping)
df['55_label'] = a
b = df['56'].map(mapping2)
df['56_label'] = b

vocab_size = 9443
max_len = 55

X = df.drop(['55', '56', '55_label','56_label'],axis=1).values
y = df['55_label'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

# X_train = pad_sequences(X_train, maxlen=max_len)
# X_test = pad_sequences(X_test, maxlen=max_len)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [31]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,55_label,56_label
0,0,0,0,0,0,0,0,0,0,0,...,79,367,316,31,102,1244,pure,story,0,0
1,0,0,0,0,0,0,0,0,0,0,...,554,341,316,1057,54,2520,fantasy,story,1,0
2,0,0,0,0,0,0,0,0,0,0,...,22,3518,8,68,5037,653,drama,omnibus,2,1
3,0,0,0,0,0,0,0,0,0,0,...,44,253,6180,1191,3089,129,sports,story,3,0
4,0,0,0,0,0,0,0,0,0,0,...,2094,1,18,495,2274,261,fantasy,story,1,0


In [24]:
print(tf.__version__)

2.8.2


In [25]:
embedding_dim = 128
hidden_units = 128
num_classes = 10

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.10))

model.add(LSTM(hidden_units))

model.add(Dense(num_classes, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_genre.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./cv_model.h5', 
                                                    monitor = 'accuracy',
                                                    save_weights_only = False,
                                                    save_best_only=True)


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [26]:
import os
import tensorflow as tf

history = model.fit(X_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc, checkpoint_cb], validation_data=(X_test, y_test))

Epoch 1/30
47/47 [==============================] - ETA: 0s - loss: 2.1290 - acc: 0.1924
Epoch 1: val_acc improved from -inf to 0.21424, saving model to best_model_genre.h5


47/47 [==============================] - 48s 888ms/step - loss: 2.1290 - acc: 0.1924 - val_loss: 2.0761 - val_acc: 0.2142
Epoch 2/30
47/47 [==============================] - ETA: 0s - loss: 1.9591 - acc: 0.2885
Epoch 2: val_acc improved from 0.21424 to 0.32169, saving model to best_model_genre.h5


47/47 [==============================] - 28s 596ms/step - loss: 1.9591 - acc: 0.2885 - val_loss: 1.8963 - val_acc: 0.3217
Epoch 3/30
47/47 [==============================] - ETA: 0s - loss: 1.6399 - acc: 0.4069
Epoch 3: val_acc improved from 0.32169 to 0.33647, saving model to best_model_genre.h5


47/47 [==============================] - 32s 688ms/step - loss: 1.6399 - acc: 0.4069 - val_loss: 1.8602 - val_acc: 0.3365
Epoch 4/30
47/47 [==============================] - ETA: 0s - loss: 1.3831 - acc: 0.4963
Epoch 4: val_acc did not improve from 0.33647


47/47 [==============================] - 26s 565ms/step - loss: 1.3831 - acc: 0.4963 - val_loss: 2.0346 - val_acc: 0.3009
Epoch 5/30
47/47 [==============================] - ETA: 0s - loss: 1.2074 - acc: 0.5398
Epoch 5: val_acc did not improve from 0.33647


47/47 [==============================] - 27s 571ms/step - loss: 1.2074 - acc: 0.5398 - val_loss: 2.0892 - val_acc: 0.2606
Epoch 6/30
47/47 [==============================] - ETA: 0s - loss: 1.0623 - acc: 0.5746
Epoch 6: val_acc did not improve from 0.33647


47/47 [==============================] - 14s 292ms/step - loss: 1.0623 - acc: 0.5746 - val_loss: 2.2910 - val_acc: 0.2411
Epoch 7/30
47/47 [==============================] - ETA: 0s - loss: 0.9747 - acc: 0.6082
Epoch 7: val_acc did not improve from 0.33647


47/47 [==============================] - 14s 301ms/step - loss: 0.9747 - acc: 0.6082 - val_loss: 2.4656 - val_acc: 0.2283
Epoch 7: early stopping


모델 저장

In [11]:
# model_json = model.to_json()
# with open("./model.json", "w") as json_file : 
#     json_file.write(model_json)
# model.save_weights("./best_model.h5")
# print("Saved model to disk")

Saved model to disk


In [27]:
model.save("./best_model_cc.h5")

In [237]:
from keras.models import model_from_json
json_file = open("model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("best_model_genre.h5")
print("Loaded model from disk")

Loaded model from disk


In [28]:
model = load_model("./best_model_cc.h5")
score = model.evaluate(X_test, y_test,verbose=0)

print("%s : %.2f%%" % (model.metrics_names[1], score[1]*100))

acc : 22.83%


In [18]:
loaded_model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
loaded_model.load("./best_model_aa.h5")
# model evaluation
score = loaded_model.evaluate(X_test, y_test,verbose=0)

print("%s : %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

NameError: ignored

In [30]:
model.predict(X_test[2].reshape(1,55)).argmax()

1

In [245]:
y_test[2]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

# input

In [219]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open('/content/drive/MyDrive/devkor/wordIndex.json') as json_file:
  word_index = json.load(json_file)
  tokenizer = Tokenizer()
  tokenizer.word_index = word_index

def SentenceToToken(sentence):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    token=[]
    token.append(tokenizer.texts_to_sequences(stopwords_removed_sentence))
    token=pad_sequences(token, maxlen=55)
    token=token.reshape(1,55)
    return token

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [222]:
from konlpy.tag import Okt
okt=Okt()

In [226]:
stopwords = []

In [227]:
sentence = "고양이가 춤을 춘다."
token = SentenceToToken(sentence)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/sequence.py:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sample_shape = np.asarray(x).shape[1:]


ValueError: ignored

In [198]:
loaded_model.predict(X_test[1].reshape(1,55))

array([[2.2708373e-03, 2.3917107e-04, 9.9749005e-01]], dtype=float32)

In [48]:
import pandas as pd
import numpy as np

rating=pd.read_csv('/content/rating_aa.csv')

def SelectNIceWebtoon(f_var, g_var):
    formatOption = rating['format'] == f_var
    genreOption = rating['genre'] == g_var
    selectedWebtoon = rating[formatOption & genreOption]
    sortedWebtoon=selectedWebtoon.sort_values(['rating'])
    NiceSortedWebtoon=sortedWebtoon[-10:]
    return NiceSortedWebtoon[['title', 'rating']]

In [49]:
a = SelectNIceWebtoon(0,0)
print(a)

                  title  rating
1739  흑막 여주가 날 새엄마로 ...    9.98
883        시월드가 내게 집착한다    9.98
1218               윈터우즈    9.98
268                냐한남자    9.98
1102             오!주예수여    9.98
248      남주의 첫날밤을 가져버렸다    9.98
378             동트는 로맨스    9.98
924              썸머 브리즈    9.98
1106          오늘도 사랑스럽개    9.98
1850     [드라마원작] 치즈인더트랩    9.99
